<a href="https://colab.research.google.com/github/ithabibi/Persian-Opinion-Mining-and-Sentiment-Analysis/blob/main/use-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Persian Sentiment Analysis With Fasttext language Model and LSTM neural network
### Persian sentiment analysis step by step guide


---


so there are 3 steps we going through with each other 

## Step1 Load fasttext model

In [ ]:
!pip install pybind11==2.11.1
!pip install fasttext==0.9.2 

#!pip install keras==2.14.0
!pip install tensorflow==2.12.0 #For Deep Learning
!pip install keras==2.12.0 #A wrapper for TensorFlow for simplicity

!pip install hazm==0.7.0
!pip install pandas==1.5.3
!pip install numpy==1.23

import pandas
import random
import numpy as np
import hazm
import keras.backend as K
import fasttext 

In [ ]:
#load and unzip ELM
!rm -rf /content/cc.fa.300.bin.gz
!rm -rf /content/cc.fa.300.bin

!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
!gunzip /content/cc.fa.300.bin.gz

%time
fasttext_model = fasttext.load_model("/content/cc.fa.300.bin")

## Step 2 Load LSTM learned model

In [ ]:
#load and unzip learned model
!rm -rf /content/learned-query-sentiment-fasttext.model.zip
!rm -rf /content/learned-query-sentiment-fasttext.model

!wget https://raw.githubusercontent.com/ithabibi/Persian-Opinion-Mining-and-Sentiment-Analysis/main/learned-query-sentiment-fasttext.model.zip
!unzip /content/learned-query-sentiment-fasttext.model.zip
embedding_dim = 300 #@param {type:"integer"} #The number 300, is the dimensions of the model
max_vocab_token = 8 #@param {type:"integer"}

from keras.models import load_model

#del model  # deletes the existing model

LSTM_model = load_model('/content/learned-query-sentiment-fasttext.model')

## Step 3 enter persian text and booooom!!!

In [ ]:
#@title using model
user_text = "فعال سازی آهنگ پیشواز" #@param {type:"string"}
from IPython.core.display import display, HTML
_normalizer = hazm.Normalizer()
if not user_text=="":
  normal_text = _normalizer.normalize(user_text)
  tokenized_text = hazm.word_tokenize(normal_text)
  
  # create and Prepare three dimension tensor (1,8,300) with zero value : (1,number_of_words, dimension_of_fasttext)
  vector_text = np.zeros((1,max_vocab_token,embedding_dim),dtype=K.floatx())

  for vocabs in range(0,len(tokenized_text)):
    if vocabs >= max_vocab_token:
      break # If the comment is more than 8 words, only the first 8 words will be considered
    if tokenized_text[vocabs] not in fasttext_model.words:
      continue # If vocab does not exist in fasttext, every 300 elements of that word's vector remain zero
    
    vector_text[0, vocabs, :] = fasttext_model.get_word_vector(tokenized_text[vocabs])

  # print(vector_text.shape)
  # print(vector_text)
  result = LSTM_model.predict(vector_text) # the result has two element: [0][1] and [0][0]
  pos_percent = str(int(result[0][1]*100))+" % 😍"
  neg_percent = str(int(result[0][0]*100))+" % 🤕"
  display(HTML("<div style='text-align: center'><div style='display:inline-block'><img height='64px' width='64px' src='https://images.rawpixel.com/image_png_1000/cHJpdmF0ZS9sci9pbWFnZXMvd2Vic2l0ZS8yMDIyLTEwL3JtNTg2LWlubG92ZWZhY2UtMDFfMS1sOWQzYzlxMC5wbmc.png'/><h4>{}</h4></div> | <div style='display:inline-block'><img height='64px' width='64px' src='https://images.rawpixel.com/image_png_1000/cHJpdmF0ZS9sci9pbWFnZXMvd2Vic2l0ZS8yMDIyLTEwL3JtNTg2LWNyeWluZ2ZhY2UtMDFfMi1sOWQzYnh0MC5wbmc.png'/><h4>{}</h4></div></div>".format(pos_percent,neg_percent)))
else:
  print("Please enter your text")

## Step 3.1 enter batch persian text

In [ ]:
#@title laod test Dataset
!wget https://raw.githubusercontent.com/ithabibi/Persian-Opinion-Mining-and-Sentiment-Analysis/main/related-query-whit-lexion.csv

# load and read sentiment_tagged dataset.csv file in tاe path ./content/ in google colab. 
# this dataset include three element: Query,Score,Suggestion. Query is feature and Suggestion is label.
csv_dataset = pandas.read_csv("/content/related-query-whit-lexion.csv")

def CleanPersianText(text):
  _normalizer = hazm.Normalizer()
  text = _normalizer.normalize(text)
  return text

# Cleansing the dataset and creating a new list with two elements: "Query" and "suggestion"filde. (but without the third element: "Score")
# The new list is created by the zip Query --> x= zip(csv_dataset['Query'],csv_dataset['Suggestion'])
# valu of suggestion is 1,2,3 or positive,negative,neutral
revlist = list(map(lambda x: [CleanPersianText(x[0]),"1","2"],zip(csv_dataset['Query'],csv_dataset['Suggestion'],csv_dataset['Score'])))

# print number of element exist in positive, neutral, negative, revlist list 
print("*" * 88)
print("Total dataset count {}".format(len(revlist)))

In [ ]:
#@title Result
from IPython.core.display import display, HTML
_normalizer = hazm.Normalizer()
for item in range(0,100): #len(revlist)
  user_text = revlist[item][0]
  if not user_text=="":
    normal_text = _normalizer.normalize(user_text)
    tokenized_text = hazm.word_tokenize(normal_text)
    # create and Prepare three dimension tensor (1,8,300) with zero value : (1,number_of_words, dimension_of_fasttext)
    vector_text = np.zeros((1,max_vocab_token,embedding_dim),dtype=K.floatx())

    for vocabs in range(0,len(tokenized_text)):
      if vocabs >= max_vocab_token:
        break # If the comment is more than 8 words, only the first 8 words will be considered
      if tokenized_text[vocabs] not in fasttext_model.words:
        continue # If vocab does not exist in fasttext, every 300 elements of that word's vector remain zero
      vector_text[0, vocabs, :] = fasttext_model.get_word_vector(tokenized_text[vocabs])
    
    result = LSTM_model.predict(vector_text, verbose='0',workers=10,use_multiprocessing=True,max_queue_size=100) # the result has two element: [0][1] and [0][0]
    pos_percent = str(int(result[0][1]*100))
    neg_percent = str(int(result[0][0]*100))
    #display(HTML("<div style='text-align: center'><div style='display:inline-block'><img height='64px' width='64px' src='https://images.rawpixel.com/image_png_1000/cHJpdmF0ZS9sci9pbWFnZXMvd2Vic2l0ZS8yMDIyLTEwL3JtNTg2LWlubG92ZWZhY2UtMDFfMS1sOWQzYzlxMC5wbmc.png'/><h4>{}</h4></div> | <div style='display:inline-block'><img height='64px' width='64px' src='https://images.rawpixel.com/image_png_1000/cHJpdmF0ZS9sci9pbWFnZXMvd2Vic2l0ZS8yMDIyLTEwL3JtNTg2LWNyeWluZ2ZhY2UtMDFfMi1sOWQzYnh0MC5wbmc.png'/><h4>{}</h4></div></div>".format(pos_percent,neg_percent)))
    print(str(item) + ": " + pos_percent +"%😍" +" " + neg_percent +"%🤕" + " " + revlist[item][0] , "\n")
  else:
    print("Please enter your text")
print("end")

اضافه کردن فیلد احساس به رلیتد کوئری ها با روش بهبود مصرف حافظه

In [ ]:
import pandas as pd  # وارد کردن کتابخانه pandas برای کار با داده‌های جدولی
import numpy as np  # وارد کردن کتابخانه numpy برای کار با آرایه‌ها و محاسبات عددی
import hazm  # وارد کردن کتابخانه hazm برای پردازش متون فارسی
from keras import backend as K  # وارد کردن backend از Keras برای مدیریت حافظه

# بارگذاری داده‌ها از اینترنت و ذخیره در DataFrame
!wget -q https://raw.githubusercontent.com/ithabibi/Persian-Opinion-Mining-and-Sentiment-Analysis/main/merged_all_operator-data.from2009to2023B.csv
csv_dataset = pd.read_csv("/content/merged_all_operator-data.from2009to2023B.csv")  # خواندن فایل CSV به DataFrame

# تابعی برای پاکسازی متن فارسی
def CleanPersianText(text):
    _normalizer = hazm.Normalizer()  # ایجاد یک نرمالایزر برای متن فارسی
    text = _normalizer.normalize(text)  # نرمال‌سازی متن
    return text  # بازگشت متن نرمال‌شده

# ایجاد لیست revlist شامل اطلاعات پردازش‌شده
revlist = list(map(lambda x: [x[0], x[1], x[2], x[3], x[4], x[5], CleanPersianText(x[6])], 
                   zip(csv_dataset['index'], csv_dataset['top25'], csv_dataset['value'], 
                       csv_dataset['date'], csv_dataset['keyword'], csv_dataset['get_type'], 
                       csv_dataset['Query'])))  # ترکیب ستون‌های مختلف و پاکسازی متن

# نمایش تعداد کل داده‌ها
print("*" * 88)  # چاپ خطی از ستاره‌ها برای جدا کردن بخش‌ها
print("Total dataset count {}".format(len(revlist)))  # نمایش تعداد کل داده‌ها

_normalizer = hazm.Normalizer()  # ایجاد نرمالایزر برای استفاده در پردازش متن
batch_size = 1000  # تعیین اندازه دسته برای پردازش
results = []  # لیست برای ذخیره نتایج نهایی
i=1
# پردازش داده‌ها به صورت دسته‌ای
for start in range(0, len(revlist), batch_size):  # تکرار برای هر دسته از داده‌ها
    end = min(start + batch_size, len(revlist))  # تعیین انتهای دسته
    batch_items = revlist[start:end]  # انتخاب دسته فعلی
#تابع مین برای اطمینان از این که مقدار محاسبه‌شده از تعداد کل آیتم‌ها  نشود، استفاده می‌شود.
    for item in batch_items:  # پردازش هر آیتم در دسته
        user_text = item[6]  # استخراج متن کاربر از آیتم
        if user_text != "":  # بررسی اینکه متن خالی نباشد
            normal_text = _normalizer.normalize(user_text)  # نرمال‌سازی متن کاربر
            tokenized_text = hazm.word_tokenize(normal_text)  # توکنایز کردن متن نرمال‌شده

            # ایجاد تنسور سه بعدی (1,8,300) با مقدار صفر
            vector_text = np.zeros((1, max_vocab_token, embedding_dim), dtype=K.floatx())  # ایجاد یک آرایه خالی برای ذخیره وکتورهای واژه

            for vocabs in range(min(len(tokenized_text), max_vocab_token)):  # تکرار بر روی توکن‌ها تا حداکثر تعداد مجاز
                if tokenized_text[vocabs] not in fasttext_model.words:  # بررسی اینکه واژه در مدل FastText وجود داشته باشد
                    continue  # اگر واژه وجود ندارد، ادامه بده
                vector_text[0, vocabs, :] = fasttext_model.get_word_vector(tokenized_text[vocabs])  # گرفتن وکتور واژه و ذخیره آن

            result = LSTM_model.predict(vector_text, verbose=0)  # پیش‌بینی با مدل LSTM و کاهش verbosity
            
            pos_percent = str(int(result[0][1] * 100))  # درصد احساس مثبت
            neg_percent = str(int(result[0][0] * 100))  # درصد احساس منفی

            # تعیین احساس و اضافه کردن به نتایج
            sentiment = "positive" if result[0][1] > result[0][0] else "negative"  # تعیین احساس بر اساس نتایج پیش‌بینی
            item.append(sentiment)  # اضافه کردن احساس به آیتم
            results.append(item)  # اضافه کردن آیتم به لیست نتایج
            i= i+1
            print(i)  

    # آزادسازی حافظه با حذف متغیرهای غیرضروری
    del batch_items  # حذف دسته فعلی از حافظه
    K.clear_session()  # آزادسازی حافظه مدل Keras

# تبدیل نتایج به DataFrame
columns = ['Index', 'Top25', 'Value', 'Date', 'Keyword', 'Get Type', 'Comment', 'Sentiment']  # نام ستون‌ها
results_df = pd.DataFrame(results, columns=columns)  # ساخت DataFrame از نتایج

# ذخیره DataFrame به عنوان یک فایل CSV
results_df.to_csv('results_with_sentiment.csv', index=False)  # ذخیره نتایج در فایل CSV بدون شماره ایندکس

print("Results saved to results_with_sentiment.csv")  # چاپ پیام موفقیت در ذخیره‌سازی نتایج
